In [1]:
import json
import math
import re
import time
from datetime import datetime
from string import punctuation, whitespace

import emoji
import numpy as np
import pandas as pd
from natasha import (PER, Doc, MorphVocab, NamesExtractor, NewsEmbedding,
                     NewsMorphTagger, NewsNERTagger, NewsSyntaxParser,
                     Segmenter)
from nltk.corpus import stopwords

In [2]:
# df = pd.read_json("./data/reviews_texts.json")
df = pd.read_csv("./reviews_cleaned.csv")
# df_fs = pd.DataFrame(df["full_sentence"].unique(), columns=["review_full_text"])
# df_fs
# df = df_fs

df

,Unnamed: 0,review_full_text,review_rating,product,category,url,review_full_text_marked
0,0,Взяла эти ящики для рассады и не пожалела. Пл...,5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
1,1,"Отличные ящики, к качеству и упаковке претензи...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
2,2,"Быстрая доставка , все цело. И подарок положил...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
3,3,"Ящики отличные, крепкие, маленький презент вну...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
4,4,"Очень хорошие ящики, уже с отверстиями для вод...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
...,...,...,...,...,...,...,...
3283,3283,Ужасная лампа. Греет очень сильно. Надолго вкл...,1.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3284,3284,"Лампа в принципе не плохая,но с ДНаТ сложно ко...",3.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3285,3285,"Очень сильно греется, нагревается буквально за...",3.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3286,3286,"заказывал на 200вт. измерил, показывала 80 вт,...",2.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked


In [3]:
# df_temp = df.reindex(df.review_full_text.str.len().sort_values(ascending=False).index)[:100]
df_temp = df
df_temp

,Unnamed: 0,review_full_text,review_rating,product,category,url,review_full_text_marked
0,0,Взяла эти ящики для рассады и не пожалела. Пл...,5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
1,1,"Отличные ящики, к качеству и упаковке претензи...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
2,2,"Быстрая доставка , все цело. И подарок положил...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
3,3,"Ящики отличные, крепкие, маленький презент вну...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
4,4,"Очень хорошие ящики, уже с отверстиями для вод...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
...,...,...,...,...,...,...,...
3283,3283,Ужасная лампа. Греет очень сильно. Надолго вкл...,1.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3284,3284,"Лампа в принципе не плохая,но с ДНаТ сложно ко...",3.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3285,3285,"Очень сильно греется, нагревается буквально за...",3.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3286,3286,"заказывал на 200вт. измерил, показывала 80 вт,...",2.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked


In [4]:
stopwords_ru = stopwords.words("russian")
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [5]:
def get_syntax(df, row, text_cell = "text"):
    res = []
    
    def recursive(df, row, res, text_cell):
        edges = df[(df["head_id"]==row["id"])]
        edges = edges[(edges["id"] != edges["head_id"])]
        if edges.empty:
            if re.search(fr"[{punctuation}]", row[text_cell]):
                return
            res.append((row[text_cell], row["rel"], row["pos"], row["lemma"]))
            return
        
        isPrinted = False
        for index, edge in edges.iterrows():
            if edge["rel"] == "conj":
                continue
            if text_cell == "lemma":
                if edge["text"] in stopwords_ru or re.search(fr"\d|[{punctuation}]", edge["text"]):
                    continue
            if int(edge["id"].split("_")[1]) > int(row["id"].split("_")[1]) and not isPrinted:
                res.append((row[text_cell], row["rel"], row["pos"], row["lemma"]))
                isPrinted = True
            recursive(df, edge, res, text_cell)
        if not isPrinted:
            res.append((row[text_cell], row["rel"], row["pos"], row["lemma"]))
            isPrinted = True
    
    recursive(df,row, res, text_cell)
    texts = [x[0] for x in res]
    texts = re.sub(r'\s([?.!,;:"](?:\s|$))', r'\1', " ".join(texts))
    rels = [x[1] for x in res]
    poses = [x[2] for x in res]
    lemmas = [x[3] for x in res]
    return texts, rels, poses, lemmas

In [6]:
numpy_data = df_temp.to_numpy()

In [7]:
black_list = ["спасибо", "доставка", "продавец", "покупка",
              "хороший", "отличный", "очень",
              "прийти"] + stopwords_ru

In [8]:
tarr = [['VERB', 'DET', 'NOUN', 'ADP', 'NOUN'], ['взять', 'этот', 'ящик', 'для', 'рассада']]
indices = [i for i, x in enumerate(tarr[0]) if x == "NOUN"]
nouns = [tarr[1][i] for i in indices if tarr[1][i] not in black_list]
bool(nouns)

True

In [9]:
def isPos(data, pos, black_list = black_list):
    indices = [i for i, x in enumerate(data.iloc[0]) if x == pos]
    nouns = [data.iloc[1][i] for i in indices if data.iloc[1][i] not in black_list]
    return bool(nouns)

In [10]:
pd.set_option('display.max_rows', 40)

In [11]:
df_res = pd.DataFrame(columns=["text", "rels", "poses", "lemmas"])
for df_item in numpy_data:
    res = df_item[df_temp.columns.get_indexer(["review_full_text"])[0]]
    doc = Doc(res)  # Doc(res.text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    data = doc.tokens
    columns = list(doc.tokens[0].as_json.keys())
    df_natasha = pd.DataFrame(data=data, columns=columns)

    try:
        df_objects = df_natasha[
            (df_natasha["rel"] == "root")
            | (df_natasha["rel"] == "conj")
            | (df_natasha["id"] == df_natasha["head_id"])
        ]
        temp = pd.DataFrame(
            df_objects.apply(lambda row: get_syntax(df_natasha, row), axis=1)
        )
        temp[["text", "rels", "poses", "lemmas"]] = pd.DataFrame(
            list(temp[temp.columns[0]]), index=temp.index
        )
        temp = temp.drop([temp.columns[0]], axis=1)
        # result = temp[(temp["poses"].apply(lambda x: "NOUN" in x)) ]#& (temp["poses"].apply(lambda x: "ADJ" in x))]
        result = temp[
            (temp[["poses", "lemmas"]].apply(lambda x: isPos(x, "NOUN"), axis=1))
            & (temp[["poses", "lemmas"]].apply(lambda x: isPos(x, "ADJ"), axis=1))
        ]
        df_res = pd.concat([df_res, result])
    except:
        print(f"Failed to add row {res}")

df_res

Failed to add row Все кашпо отличные целые срок доставки раньше спасибо огромноепрадавцу С НАСТУПАЮЩИМ ВАС НОВЫМ ГОДОМ, ВСЕМ БОЖЬЕЙ БЛАГОДАТИ !🥰💞💋👍👍👍🙏🙏🙏🥳🎄🎄🎄
Failed to add row Заработок от 3000 в день на таких отзывах, чтобы устроится смотри фото!
Failed to add row Очень симпатичное кашпо😍качественный пластик, не хрупкий👍буду заказывать у этого продавца ещё 💯%
Failed to add row на фото горшки вроде большие на самом деле маленькие посадочные места
Failed to add row Хорошая подставка, но для маленьких горшков, под фиалки, диаметром не больше 13 см.


,text,rels,poses,lemmas
10,Пластик прочный,"[nsubj, root]","[NOUN, ADJ]","[пластик, прочный]"
12,есть съёмный поддон,"[root, amod, nsubj]","[VERB, ADJ, NOUN]","[быть, съемный, поддон]"
17,идеальные размеры,"[amod, conj]","[ADJ, NOUN]","[идеальный, размер]"
21,красивый внешний вид,"[amod, amod, conj]","[ADJ, ADJ, NOUN]","[красивый, внешний, вид]"
24,Отдельное спасибо продавцу за подарок,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]"
...,...,...,...,...
4,Лампа принципе не плохая десять дней,"[nsubj, obl, advmod, root, nummod:gov, iobj]","[NOUN, NOUN, PART, ADJ, NUM, NOUN]","[лампа, принцип, не, плохой, десять, день]"
9,но сложно в с ДНаТ конкурировать,"[cc, conj, case, case, obl, csubj]","[CCONJ, ADJ, ADP, ADP, NOUN, VERB]","[но, сложный, в, с, днат, конкурировать]"
12,так и должно быть или это брак,"[cc, fixed, conj, xcomp, cc, nsubj, nsubj]","[ADV, PART, ADJ, VERB, CCONJ, PRON, NOUN]","[так, и, должный, быть, или, это, брак]"
5,измерил на следующей день,"[root, case, amod, obl]","[VERB, ADP, ADJ, NOUN]","[измерить, на, следующий, день]"


In [12]:
". ".join(df_res["text"])

'Пластик прочный. есть съёмный поддон. идеальные размеры. красивый внешний вид. Отдельное спасибо продавцу за подарок. Спасибо за маленький сюрприз как внимание к покупателю. Быстрая доставка все цело. Небольшой подарок. Невооружённым взглядом были видны сколы. Почему такая транспортировка отвратительная. Маленький презент был сюрпризом Рекомендую к покупке. Подойдёт для выращивания на подоконнике зелени. растения будут рады. все целое отдельное спасибо за подарок внутри. Упаковка в прозрачные пакеты. Купилась на отзывы что идеально для всех балконов. Но оказалось не для моего у меня плоские перила. к полной маскировке закрепляла для удобства ухода на внутреннюю сторону балкона Компактный. что само крепление не тяжёлое. и может варьироваться на разную ширину перил. Важно то что до конечного потребителя приходит некачественный товар. Продавцу надо либо продумывать более надежный вариант упаковки проблема в грузчиках либо отслеживать то если что находится на складах. Отличные ящики очень

In [13]:
df_test = df_res.reset_index()
df_test = df_test.drop(["index"], axis=1)
df_test

,text,rels,poses,lemmas
0,Пластик прочный,"[nsubj, root]","[NOUN, ADJ]","[пластик, прочный]"
1,есть съёмный поддон,"[root, amod, nsubj]","[VERB, ADJ, NOUN]","[быть, съемный, поддон]"
2,идеальные размеры,"[amod, conj]","[ADJ, NOUN]","[идеальный, размер]"
3,красивый внешний вид,"[amod, amod, conj]","[ADJ, ADJ, NOUN]","[красивый, внешний, вид]"
4,Отдельное спасибо продавцу за подарок,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]"
...,...,...,...,...
4214,Лампа принципе не плохая десять дней,"[nsubj, obl, advmod, root, nummod:gov, iobj]","[NOUN, NOUN, PART, ADJ, NUM, NOUN]","[лампа, принцип, не, плохой, десять, день]"
4215,но сложно в с ДНаТ конкурировать,"[cc, conj, case, case, obl, csubj]","[CCONJ, ADJ, ADP, ADP, NOUN, VERB]","[но, сложный, в, с, днат, конкурировать]"
4216,так и должно быть или это брак,"[cc, fixed, conj, xcomp, cc, nsubj, nsubj]","[ADV, PART, ADJ, VERB, CCONJ, PRON, NOUN]","[так, и, должный, быть, или, это, брак]"
4217,измерил на следующей день,"[root, case, amod, obl]","[VERB, ADP, ADJ, NOUN]","[измерить, на, следующий, день]"


In [14]:
df_test.reindex(df_test.text.str.len().sort_values(ascending=True).index)[2000:2100]

,text,rels,poses,lemmas
2420,и без повреждений за быструю поставку,"[advmod, case, conj, case, amod, nmod]","[CCONJ, ADP, NOUN, ADP, ADJ, NOUN]","[и, без, повреждение, за, быстрый, поставка]"
1147,Полный комплект все хорошего качества,"[amod, nsubj, nsubj, amod, conj]","[ADJ, NOUN, PRON, ADJ, NOUN]","[полный, комплект, весь, хороший, качество]"
2191,Очень красивое кашпо хорошего размера,"[advmod, amod, root, amod, nmod]","[ADV, ADJ, NOUN, ADJ, NOUN]","[очень, красивый, кашпо, хороший, размер]"
1158,Стойка хорошая все детали в комплекте,"[nsubj, root, det, nsubj, case, nmod]","[ADJ, ADJ, DET, NOUN, ADP, NOUN]","[стойка, хороший, весь, деталь, в, комплект]"
2715,Родставку купила белую в подарок маме,"[nsubj, root, obj, case, nmod, iobj]","[PROPN, VERB, ADJ, ADP, NOUN, NOUN]","[родставка, купить, белый, в, подарок, мама]"
...,...,...,...,...
2635,Доставка быстрая целая без повреждений,"[nsubj, amod, conj, case, nmod]","[NOUN, ADJ, ADJ, ADP, NOUN]","[доставка, быстрый, целый, без, повреждение]"
2917,Отличная подставка я думала будет хуже,"[amod, nsubj, nsubj, advcl, cop, advcl]","[ADJ, NOUN, PRON, VERB, AUX, ADJ]","[отличный, подставка, я, думать, быть, хуже]"
1292,самый верхний это почти уже косяк вб 😣,"[amod, amod, nsubj, advmod, advmod, amod, root...","[ADJ, ADJ, PRON, ADV, ADV, NOUN, NOUN, PUNCT]","[самый, верхний, это, почти, уже, косяк, вб, 😣]"
105,яркие пятна видимо но вид это не портит,"[amod, nsubj, advmod, cc, nsubj, nsubj, advmod...","[ADJ, NOUN, ADV, CCONJ, NOUN, PRON, PART, VERB]","[яркий, пятно, видимо, но, вид, это, не, портить]"


In [15]:
df_test["raw"] = [list(zip(*l)) for l in list(zip(df_test.lemmas, df_test.poses, df_test.rels))]
df_test

,text,rels,poses,lemmas,raw
0,Пластик прочный,"[nsubj, root]","[NOUN, ADJ]","[пластик, прочный]","[(пластик, NOUN, nsubj), (прочный, ADJ, root)]"
1,есть съёмный поддон,"[root, amod, nsubj]","[VERB, ADJ, NOUN]","[быть, съемный, поддон]","[(быть, VERB, root), (съемный, ADJ, amod), (по..."
2,идеальные размеры,"[amod, conj]","[ADJ, NOUN]","[идеальный, размер]","[(идеальный, ADJ, amod), (размер, NOUN, conj)]"
3,красивый внешний вид,"[amod, amod, conj]","[ADJ, ADJ, NOUN]","[красивый, внешний, вид]","[(красивый, ADJ, amod), (внешний, ADJ, amod), ..."
4,Отдельное спасибо продавцу за подарок,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]","[(отдельный, ADJ, amod), (спасибо, NOUN, root)..."
...,...,...,...,...,...
4214,Лампа принципе не плохая десять дней,"[nsubj, obl, advmod, root, nummod:gov, iobj]","[NOUN, NOUN, PART, ADJ, NUM, NOUN]","[лампа, принцип, не, плохой, десять, день]","[(лампа, NOUN, nsubj), (принцип, NOUN, obl), (..."
4215,но сложно в с ДНаТ конкурировать,"[cc, conj, case, case, obl, csubj]","[CCONJ, ADJ, ADP, ADP, NOUN, VERB]","[но, сложный, в, с, днат, конкурировать]","[(но, CCONJ, cc), (сложный, ADJ, conj), (в, AD..."
4216,так и должно быть или это брак,"[cc, fixed, conj, xcomp, cc, nsubj, nsubj]","[ADV, PART, ADJ, VERB, CCONJ, PRON, NOUN]","[так, и, должный, быть, или, это, брак]","[(так, ADV, cc), (и, PART, fixed), (должный, A..."
4217,измерил на следующей день,"[root, case, amod, obl]","[VERB, ADP, ADJ, NOUN]","[измерить, на, следующий, день]","[(измерить, VERB, root), (на, ADP, case), (сле..."


In [16]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import pickle

tokenizer = RegexTokenizer()
dostoevsky_model = FastTextSocialNetworkModel(tokenizer=tokenizer)
loaded_model = pickle.load(open("./StreamLit/model.sav", "rb"))

probs = loaded_model.predict_proba(df_test["lemmas"].str.join(" ").values)
predictions = loaded_model.predict(df_test["lemmas"].str.join(" ").values)
df_test["prediction"] = predictions

messages = df_test["text"]
results = dostoevsky_model.predict(messages, k=5)

temp_arr = []
for tones in results:
    p = tones["positive"]
    n = tones["negative"]
    if p > n:
        temp_arr.append("positive")
    else:
        temp_arr.append("negative")
df_test["tone"] = temp_arr
df_test["sentiment"] = [{k: round(v, 2) for k, v in dct.items()} for dct in results]
df_test["proba"] = [x[2] for x in probs]

df_test_usefull = df_test[df_test["prediction"] == "Содержательные"]

In [17]:
df_test = df_test_usefull
df_test

,text,rels,poses,lemmas,raw,prediction,tone,sentiment,proba
0,Пластик прочный,"[nsubj, root]","[NOUN, ADJ]","[пластик, прочный]","[(пластик, NOUN, nsubj), (прочный, ADJ, root)]",Содержательные,positive,"{'skip': 0.09, 'neutral': 0.09, 'positive': 0....",0.807817
2,идеальные размеры,"[amod, conj]","[ADJ, NOUN]","[идеальный, размер]","[(идеальный, ADJ, amod), (размер, NOUN, conj)]",Содержательные,negative,"{'neutral': 0.99, 'negative': 0.09, 'positive'...",0.599696
4,Отдельное спасибо продавцу за подарок,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]","[(отдельный, ADJ, amod), (спасибо, NOUN, root)...",Содержательные,positive,"{'speech': 0.73, 'neutral': 0.13, 'skip': 0.01...",0.825900
7,Небольшой подарок,"[amod, root]","[ADJ, NOUN]","[небольшой, подарок]","[(небольшой, ADJ, amod), (подарок, NOUN, root)]",Содержательные,positive,"{'neutral': 0.97, 'skip': 0.01, 'positive': 0....",0.828954
13,все целое отдельное спасибо за подарок внутри,"[det, amod, amod, conj, case, nmod, advmod]","[DET, ADJ, ADJ, NOUN, ADP, NOUN, ADV]","[весь, целый, отдельный, спасибо, за, подарок,...","[(весь, DET, det), (целый, ADJ, amod), (отдель...",Содержательные,negative,"{'speech': 0.87, 'skip': 0.07, 'neutral': 0.03...",0.825900
...,...,...,...,...,...,...,...,...,...
4119,Без проблем прицепила к ручке окна как необходимо,"[case, obl, root, case, obl, nmod, mark, advcl]","[ADP, NOUN, NOUN, ADP, NOUN, NOUN, SCONJ, ADJ]","[без, проблема, прицепить, к, ручка, окно, как...","[(без, ADP, case), (проблема, NOUN, obl), (при...",Содержательные,positive,"{'positive': 0.31, 'neutral': 0.21, 'skip': 0....",0.927310
4129,и один минус сетевой адаптер даже не знаю в че...,"[advmod, nummod, nsubj, amod, nsubj, advmod, a...","[PART, NUM, NOUN, ADJ, NOUN, PART, PART, VERB,...","[и, один, минус, сетевой, адаптер, даже, не, з...","[(и, PART, advmod), (один, NUM, nummod), (мину...",Содержательные,positive,"{'neutral': 0.95, 'skip': 0.07, 'positive': 0....",0.518902
4133,Есть одно но которое сводит на нет в первую оч...,"[root, nummod, advmod, nsubj, acl:relcl, case,...","[VERB, DET, NOUN, PRON, VERB, ADP, PART, ADP, ...","[быть, один, но, который, сводить, на, нет, в,...","[(быть, VERB, root), (один, DET, nummod), (но,...",Содержательные,negative,"{'neutral': 0.69, 'negative': 0.19, 'positive'...",0.937944
4179,Охватывает большой радиус освещения,"[root, amod, obj, nmod]","[VERB, ADJ, NOUN, NOUN]","[охватывать, большой, радиус, освещение]","[(охватывать, VERB, root), (большой, ADJ, amod...",Содержательные,positive,"{'neutral': 0.66, 'speech': 0.04, 'positive': ...",0.780721


In [18]:
nouns = pd.Series(
    [
        item[0]
        for sublist in df_test["raw"]
        for item in sublist
        if not (item[0] in stopwords_ru or re.search(rf"\d|[{punctuation}]", item[0]))
        and item[1] == "NOUN"
    ]
).value_counts()
nouns

цена       72
подарок    70
пластик    65
цвет       41
размер     37
           ..
ель         1
этап        1
грамм       1
литраж      1
радиус      1
Name: count, Length: 416, dtype: int64

In [19]:
nouns[nouns>5]

цена           72
подарок        70
пластик        65
цвет           41
размер         37
спасибо        35
минус          23
продавец       20
отверстие      20
горшок         19
качество       16
кашпо          13
товар          13
дизайн         13
ручка          12
инструкция     11
конструкция    11
интерьер       11
сборка         10
комплект       10
горшочек       10
лампа          10
подставка       9
фото            8
ящик            8
год             8
коробка         7
растение        7
упаковка        7
вода            7
окно            7
сторона         6
цветок          6
высота          6
Name: count, dtype: int64

In [20]:
df_test_nouns = df_test
df_test_nouns["noun"] = df_test_nouns["raw"].apply(lambda x: 
    [
        item[0]
        for item in x
        if not (item[0] in stopwords_ru or re.search(rf"\d|[{punctuation}]", item[0]))
        and item[1] == "NOUN"
    ])
df_test_nouns = df_test_nouns.explode("noun")
df_test_nouns

C:\Users\Kirill\AppData\Local\Temp\ipykernel_7004\1125050583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_nouns["noun"] = df_test_nouns["raw"].apply(lambda x:


,text,rels,poses,lemmas,raw,prediction,tone,sentiment,proba,noun
0,Пластик прочный,"[nsubj, root]","[NOUN, ADJ]","[пластик, прочный]","[(пластик, NOUN, nsubj), (прочный, ADJ, root)]",Содержательные,positive,"{'skip': 0.09, 'neutral': 0.09, 'positive': 0....",0.807817,пластик
2,идеальные размеры,"[amod, conj]","[ADJ, NOUN]","[идеальный, размер]","[(идеальный, ADJ, amod), (размер, NOUN, conj)]",Содержательные,negative,"{'neutral': 0.99, 'negative': 0.09, 'positive'...",0.599696,размер
4,Отдельное спасибо продавцу за подарок,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]","[(отдельный, ADJ, amod), (спасибо, NOUN, root)...",Содержательные,positive,"{'speech': 0.73, 'neutral': 0.13, 'skip': 0.01...",0.825900,спасибо
4,Отдельное спасибо продавцу за подарок,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]","[(отдельный, ADJ, amod), (спасибо, NOUN, root)...",Содержательные,positive,"{'speech': 0.73, 'neutral': 0.13, 'skip': 0.01...",0.825900,продавец
4,Отдельное спасибо продавцу за подарок,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]","[(отдельный, ADJ, amod), (спасибо, NOUN, root)...",Содержательные,positive,"{'speech': 0.73, 'neutral': 0.13, 'skip': 0.01...",0.825900,подарок
...,...,...,...,...,...,...,...,...,...,...
4133,Есть одно но которое сводит на нет в первую оч...,"[root, nummod, advmod, nsubj, acl:relcl, case,...","[VERB, DET, NOUN, PRON, VERB, ADP, PART, ADP, ...","[быть, один, но, который, сводить, на, нет, в,...","[(быть, VERB, root), (один, DET, nummod), (но,...",Содержательные,negative,"{'neutral': 0.69, 'negative': 0.19, 'positive'...",0.937944,фитосветильник
4179,Охватывает большой радиус освещения,"[root, amod, obj, nmod]","[VERB, ADJ, NOUN, NOUN]","[охватывать, большой, радиус, освещение]","[(охватывать, VERB, root), (большой, ADJ, amod...",Содержательные,positive,"{'neutral': 0.66, 'speech': 0.04, 'positive': ...",0.780721,радиус
4179,Охватывает большой радиус освещения,"[root, amod, obj, nmod]","[VERB, ADJ, NOUN, NOUN]","[охватывать, большой, радиус, освещение]","[(охватывать, VERB, root), (большой, ADJ, amod...",Содержательные,positive,"{'neutral': 0.66, 'speech': 0.04, 'positive': ...",0.780721,освещение
4186,Высота подходящая и для маленьких растений,"[nsubj, root, cc, case, amod, obl]","[NOUN, ADJ, PART, ADP, ADJ, NOUN]","[высота, подходящий, и, для, маленький, растение]","[(высота, NOUN, nsubj), (подходящий, ADJ, root...",Содержательные,positive,"{'neutral': 0.45, 'skip': 0.18, 'positive': 0....",0.620615,высота


In [21]:
df_x = df_test_nouns.groupby(["noun"])["text"].agg({
    ". ".join,
})
df_x.columns = [col[0] for col in df_x.columns]
df_x

,j
noun,
d,Легким движением руки бабочки превращаются 3 в...
адаптер,и адаптер оказался бракованным. и один минус с...
аддекватная,И цена самая аддекватная
бабочка,👍😊😊😊👍 бабочки на них очень легко отгибаются. м...
бархотка,Хороший прочный пластик с поддоном Уже посадил...
...,...
этап,на любом этапе транспортировки Покупала Большо...
этикетка,Легко отклеиваются этикетки
эффект,Эффект такой же как от обычной фитолампы так к...


In [22]:
df_x[df_x.columns[0]].agg(len).sort_values(ascending=False)

C:\Users\Kirill\AppData\Local\Temp\ipykernel_7004\1417927509.py:1: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df_x[df_x.columns[0]].agg(len).sort_values(ascending=False)


noun
подарок      3979
цена         3605
пластик      2888
цвет         2036
спасибо      1913
             ... 
ок             20
завернуть      19
кривой         17
то🕊            16
боковина       14
Name: j, Length: 416, dtype: int64

In [23]:
df_nouns = pd.DataFrame(df_x[df_x.columns[0]])
df_nouns["len"] = df_nouns["j"].agg(len)
df_nouns = df_nouns.sort_values("len", ascending=False)
df_nouns

C:\Users\Kirill\AppData\Local\Temp\ipykernel_7004\2401856422.py:2: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df_nouns["len"] = df_nouns["j"].agg(len)


,j,len
noun,,
подарок,Отдельное спасибо продавцу за подарок. Небольш...,3979
цена,но цена стала выше. Добрый день удивило что дн...,3605
пластик,Пластик прочный. Пластик очень толстый. Плотны...,2888
цвет,ещё кашпо в цвет качество супер пластик. Цвет ...,2036
спасибо,Отдельное спасибо продавцу за подарок. все цел...,1913
...,...,...
ок,легко Все ок собрать,20
завернуть,Устойчивый завернут,19
кривой,целый весь кривой,17


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
corpus = [df_nouns["j"]["пластик"]]
corpus

['Пластик прочный. Пластик очень толстый. Плотный пластик. Хороший качественный пластик. Отличные ящики из прочного пластика с поддоном. пластик не тонкий. Большая просьба к продавцу упаковывайте пожалуйста лучше пластик довольно хрупкий. и общее впечатление Ну пластик лучшего качества не толстый выдержит пару сезонов. сделаны из более тонкого пластика. пластик остался как новый. пластик прочный. Плотный пластик. Правда есть небольшая деформация пластика. Хороший прочный пластик с поддоном Уже посадила в него бархотки. Из минусов Пластик хрупкий 20 лет провёл на солнце. Пластик прочный в то же время. Из плотного пластика. Пластик качественный. Удобная конструкция пластик не хлипкий. Достаточно мягкий пластик. Пластик прочный корни дышали как будут смотреться цветы в нём чтоб изумительно. Но ради чистой планеты хорошо было бы в коробку упаковывать или в перерабатываемый пластик. пластик достаточно твёрдый самый раз. Пластик не тонкий. Нигде ничего не протекает плотный матовый пластик. У

In [26]:
regex = re.compile(r"(\sне)\s", re.IGNORECASE)
corpus = regex.sub(r"\g<1>", corpus[0])
corpus

'Пластик прочный. Пластик очень толстый. Плотный пластик. Хороший качественный пластик. Отличные ящики из прочного пластика с поддоном. пластик нетонкий. Большая просьба к продавцу упаковывайте пожалуйста лучше пластик довольно хрупкий. и общее впечатление Ну пластик лучшего качества нетолстый выдержит пару сезонов. сделаны из более тонкого пластика. пластик остался как новый. пластик прочный. Плотный пластик. Правда есть небольшая деформация пластика. Хороший прочный пластик с поддоном Уже посадила в него бархотки. Из минусов Пластик хрупкий 20 лет провёл на солнце. Пластик прочный в то же время. Из плотного пластика. Пластик качественный. Удобная конструкция пластик нехлипкий. Достаточно мягкий пластик. Пластик прочный корни дышали как будут смотреться цветы в нём чтоб изумительно. Но ради чистой планеты хорошо было бы в коробку упаковывать или в перерабатываемый пластик. пластик достаточно твёрдый самый раз. Пластик нетонкий. Нигде ничего непротекает плотный матовый пластик. Ужасный

In [27]:
doc_corpus = Doc(corpus)
doc_corpus.segment(segmenter)
doc_corpus.tag_morph(morph_tagger)
doc_corpus.parse_syntax(syntax_parser)
doc_corpus.tag_morph(morph_tagger)
for token in doc_corpus.tokens:
    token.lemmatize(morph_vocab)
data_natasha = doc_corpus.tokens
columns = list(doc_corpus.tokens[0].as_json.keys())
df_natasha = pd.DataFrame(data=data_natasha, columns=columns)
df_natasha

,start,stop,text,id,head_id,rel,pos,feats,lemma
0,0,7,Пластик,1_1,1_2,nsubj,NOUN,"{'Animacy': 'Inan', 'Case': 'Nom', 'Gender': '...",пластик
1,8,15,прочный,1_2,1_0,root,ADJ,"{'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Ma...",прочный
2,15,16,.,1_3,1_2,punct,PUNCT,{},.
3,17,24,Пластик,2_1,2_3,nsubj,NOUN,"{'Animacy': 'Inan', 'Case': 'Nom', 'Gender': '...",пластик
4,25,30,очень,2_2,2_3,advmod,ADV,{'Degree': 'Pos'},очень
...,...,...,...,...,...,...,...,...,...
447,2834,2838,имел,48_9,48_4,conj,VERB,"{'Aspect': 'Imp', 'Gender': 'Masc', 'Mood': 'I...",иметь
448,2839,2846,сильный,48_10,48_11,amod,ADJ,"{'Animacy': 'Inan', 'Case': 'Acc', 'Degree': '...",сильный
449,2847,2852,запах,48_11,48_9,obj,NOUN,"{'Animacy': 'Inan', 'Case': 'Acc', 'Gender': '...",запах
450,2853,2861,горелого,48_12,48_11,nmod,ADJ,"{'Case': 'Gen', 'Degree': 'Pos', 'Gender': 'Ma...",горелый


In [28]:
df_adj = df_natasha[df_natasha["pos"] == "ADJ"]
df_adj

,start,stop,text,id,head_id,rel,pos,feats,lemma
1,8,15,прочный,1_2,1_0,root,ADJ,"{'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Ma...",прочный
5,31,38,толстый,2_3,2_0,root,ADJ,"{'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Ma...",толстый
7,40,47,Плотный,3_1,3_2,amod,ADJ,"{'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Ma...",плотный
10,57,64,Хороший,4_1,4_3,amod,ADJ,"{'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Ma...",хороший
11,65,77,качественный,4_2,4_3,amod,ADJ,"{'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Ma...",качественный
...,...,...,...,...,...,...,...,...,...
433,2753,2761,защитном,47_9,47_10,amod,ADJ,"{'Case': 'Loc', 'Degree': 'Pos', 'Gender': 'Ne...",защитный
435,2771,2776,самих,47_11,47_12,amod,ADJ,"{'Case': 'Gen', 'Degree': 'Pos', 'Number': 'Pl...",сам
440,2799,2808,химозного,48_2,48_3,amod,ADJ,"{'Case': 'Gen', 'Degree': 'Pos', 'Gender': 'Ma...",химозный
448,2839,2846,сильный,48_10,48_11,amod,ADJ,"{'Animacy': 'Inan', 'Case': 'Acc', 'Degree': '...",сильный


In [29]:
cv = CountVectorizer()   
cv_fit = cv.fit_transform(df_adj['lemma'])    
word_list = cv.get_feature_names_out() 
count_list = cv_fit.toarray().sum(axis=0)
df_counts = pd.DataFrame.from_dict(zip(word_list,count_list))
df_counts = df_counts.rename({0: 'token', 1: 'count'}, axis=1)
df_counts = df_counts.sort_values(["count"], ascending=False)
df_counts = df_counts[df_counts["count"] > 1]
df_counts.to_dict('tight')["data"]

[['прочный', 10],
 ['плотный', 6],
 ['хороший', 5],
 ['качественный', 4],
 ['тонкий', 4],
 ['крепкий', 4],
 ['мягкий', 3],
 ['дешевый', 3],
 ['красивый', 3],
 ['классный', 2],
 ['сам', 2],
 ['твердый', 2],
 ['толстый', 2],
 ['неочень', 2],
 ['другой', 2],
 ['хрупкий', 2],
 ['горшок', 2]]

In [30]:
def get_counts(corpus):
    try:
        regex = re.compile(r"(\sне)\s", re.IGNORECASE)
        corpus = regex.sub(r"\g<1>", corpus)
        
        doc_corpus = Doc(corpus)
        doc_corpus.segment(segmenter)
        doc_corpus.tag_morph(morph_tagger)
        doc_corpus.parse_syntax(syntax_parser)
        doc_corpus.tag_morph(morph_tagger)
        for token in doc_corpus.tokens:
            token.lemmatize(morph_vocab)
        data_natasha = doc_corpus.tokens
        columns = list(doc_corpus.tokens[0].as_json.keys())
        df_natasha = pd.DataFrame(data=data_natasha, columns=columns)
        
        df_adj = df_natasha[df_natasha["pos"] == "ADJ"]
        if len(df_adj.index) == 0:
            return []
        
        cv = CountVectorizer()   
        cv_fit = cv.fit_transform(df_adj['lemma'])    
        word_list = cv.get_feature_names_out() 
        count_list = cv_fit.toarray().sum(axis=0)
        df_counts = pd.DataFrame.from_dict(zip(word_list,count_list))
        df_counts = df_counts.rename({0: 'token', 1: 'count'}, axis=1)
        df_counts = df_counts.sort_values(["count"], ascending=False)
        df_counts = df_counts[df_counts["count"] > 2]
        
        return df_counts.to_dict('tight')["data"]
    except:
        return []

In [31]:
pd.set_option('display.max_rows', 20)

In [32]:
df_nouns

,j,len
noun,,
подарок,Отдельное спасибо продавцу за подарок. Небольш...,3979
цена,но цена стала выше. Добрый день удивило что дн...,3605
пластик,Пластик прочный. Пластик очень толстый. Плотны...,2888
цвет,ещё кашпо в цвет качество супер пластик. Цвет ...,2036
спасибо,Отдельное спасибо продавцу за подарок. все цел...,1913
...,...,...
ок,легко Все ок собрать,20
завернуть,Устойчивый завернут,19
кривой,целый весь кривой,17


In [33]:
noun_lemmas = df_nouns["j"].agg(get_counts)
noun_lemmas

C:\Users\Kirill\AppData\Local\Temp\ipykernel_7004\740963691.py:1: FutureWarning: using <function get_counts at 0x0000014BF8DC2790> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  noun_lemmas = df_nouns["j"].agg(get_counts)


noun
подарок      [[отдельный, 12], [небольшой, 5], [отличный, 5...
цена         [[хороший, 5], [высокий, 5], [приятный, 5], [н...
пластик      [[прочный, 10], [плотный, 6], [хороший, 5], [к...
цвет                                             [[белый, 24]]
спасибо      [[отдельный, 16], [большой, 5], [огромный, 3],...
                                   ...                        
ок                                                          []
завернуть                                                   []
кривой                                                      []
то🕊                                                         []
боковина                                                    []
Name: j, Length: 416, dtype: object

In [34]:
df_noun_words_counts = pd.DataFrame(noun_lemmas)
df_noun_words_counts

,j
noun,
подарок,"[[отдельный, 12], [небольшой, 5], [отличный, 5..."
цена,"[[хороший, 5], [высокий, 5], [приятный, 5], [н..."
пластик,"[[прочный, 10], [плотный, 6], [хороший, 5], [к..."
цвет,"[[белый, 24]]"
спасибо,"[[отдельный, 16], [большой, 5], [огромный, 3],..."
...,...
ок,[]
завернуть,[]
кривой,[]


In [35]:
df_noun_words_counts["j"].agg(len) > 2

C:\Users\Kirill\AppData\Local\Temp\ipykernel_7004\1934830910.py:1: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df_noun_words_counts["j"].agg(len) > 2


noun
подарок       True
цена          True
пластик       True
цвет         False
спасибо       True
             ...  
ок           False
завернуть    False
кривой       False
то🕊          False
боковина     False
Name: j, Length: 416, dtype: bool

In [36]:
pd.set_option('max_colwidth', 400)
pd.set_option('display.max_rows', 25)

In [37]:
df_noun_words_counts[df_noun_words_counts["j"].agg(len) > 0]

C:\Users\Kirill\AppData\Local\Temp\ipykernel_7004\3500821967.py:1: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df_noun_words_counts[df_noun_words_counts["j"].agg(len) > 0]


,j
noun,
подарок,"[[отдельный, 12], [небольшой, 5], [отличный, 5], [самый, 4], [новый, 4], [новогодний, 4], [большой, 4], [приятный, 3], [хороший, 3], [целый, 3]]"
цена,"[[хороший, 5], [высокий, 5], [приятный, 5], [ниже, 4], [отличный, 4], [выгодный, 4], [самый, 4], [выше, 3], [данный, 3]]"
пластик,"[[прочный, 10], [плотный, 6], [хороший, 5], [качественный, 4], [тонкий, 4], [крепкий, 4], [мягкий, 3], [дешевый, 3], [красивый, 3]]"
цвет,"[[белый, 24]]"
спасибо,"[[отдельный, 16], [большой, 5], [огромный, 3], [подарочек, 3]]"
продавец,"[[отдельный, 3]]"
отверстие,"[[дренажный, 4]]"
минус,"[[единственный, 10], [большой, 3]]"
размер,"[[маленький, 4], [идеальный, 3], [самый, 3]]"
